## Pipeline:
Pipelines chains together multiple steps so that the output of each step is used as input to the next step.<br>
Pipelines makes it easy to apply the same preprocessing to train and test.

পাইপলাইন (Pipeline) হল একটি স্বয়ংক্রিয় ও সংগঠিত ওয়ার্কফ্লো, যেখানে ডেটা প্রসেসিং, মডেল ট্রেনিং এবং প্রেডিকশন ধাপে ধাপে স্বয়ংক্রিয়ভাবে পরিচালিত হয়। পাইপলাইন ব্যবহার করলে কোড ম্যানেজমেন্ট সহজ হয় এবং পুনরায় ব্যবহারযোগ্য (reusable) মডেল তৈরি করা যায়।<br>
### Pipeline তৈরির কারণ
1. Code Reusability – বারবার কোড লিখতে হয় না
2. Automation – প্রতিবার ম্যানুয়ালি কাজ করতে হয় না
3. Efficiency – সময় ও প্রসেসিং পাওয়ার বাঁচায়
4. Consistency – সব সময় নির্ভুল ফলাফল দেয় <br>

ডেটা সংগ্রহ  → প্রিপ্রসেসিং(Remove missing values using simpleimputer) → ফিচার ইঞ্জিনিয়ারিং(OneHotencoding) -> Feature seletion best 5 to 10 → মডেল ট্রেনিং → মূল্যায়ন → ডিপ্লয়মেন্ট

In [2]:
import numpy as np
import pandas as pd

### Import all from sklearn
1. train_test_split -> fro train and test dataset
2. ClumnTransformer - use it to transform columns
3. SimpleImputer - > use it for missing values fill.
4. OneHotEncoder - use it for new columns or featurs create
5. MinMaxscaler -- convert columns into scaling 0 to 1
6. Pipeline, make_piple -> use it dor doing all this in a line 
7. SelectkBest -- best or top 5to 10 fetures selection
8. DecsiionTree- use this algorithm for model 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier